In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

In [ ]:
from tensorflow.keras.models import load_model

model1 = load_model("/content/drive/My Drive/stacked-bi-lstm.h5")
model2 = load_model("/content/drive/My Drive/stacked-bi-gru.h5")
model3 = load_model("/content/drive/My Drive/parallel-bi-lstm-bi-gru.h5")


In [ ]:
import pandas as pd
import json
import os

# Load dataset from JSONL file
def load_dataset(file_path):
    dataset = []
    with open(file_path, 'r') as file:
        for line in file:
            example = json.loads(line)
            dataset.append(example)
    return dataset

# Define file paths
train_file = '/content/drive/My Drive/Hateful-memes/data/train.jsonl'
image_directory = '/content/drive/My Drive/Hateful-memes/data/'

# Load train dataset
train_dataset = load_dataset(train_file)

chosen_examples = train_dataset[8001:]

# Create DataFrame
data = []
for idx, example in enumerate(chosen_examples):
    text = example['text']
    img_path = os.path.join(image_directory, example['img'])
    label = example['label']
    data.append({'id': idx, 'img_text': text, 'img_path': img_path, 'label': label})

df = pd.DataFrame(data)

# Display DataFrame
print(df.head())


   id                                           img_text  \
0   0      charlie sheen after his tigers blood kicks in   
1   1       when you've finished with your life of crime   
2   2           who would win? ching-chongs one eggy boi   
3   3                   meanwhile at the isis strip club   
4   4  enough with the nazi jokes, they're making me ...   

                                            img_path  label  
0  /content/drive/My Drive/Hateful-memes/data/img...      0  
1  /content/drive/My Drive/Hateful-memes/data/img...      0  
2  /content/drive/My Drive/Hateful-memes/data/img...      1  
3  /content/drive/My Drive/Hateful-memes/data/img...      0  
4  /content/drive/My Drive/Hateful-memes/data/img...      0  


In [ ]:
print(df.shape)
df=df.head(4000)

(499, 4)


In [ ]:
print(df.shape)

(499, 4)


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.keras.models import Model

# Remove the second-to-last layer (the dropout layer)
feature_extractor_model = Model(inputs=model1.input, outputs=model1.layers[-3].output)

In [ ]:
# Assuming you have a function to preprocess the text into the required format
def preprocess_text(text):
    text = re.sub(r'[^A-Za-z]', ' ', text)
    text = text.lower()
    return text
tokenizer = Tokenizer()

In [ ]:
def extract_features(df, feature_extractor_model, tokenizer, max_sequence_length=128):
    preprocessed_texts = [preprocess_text(text) for text in df['img_text']]
    tokenized_texts = tokenizer.texts_to_sequences(preprocessed_texts)
    padded_texts = pad_sequences(tokenized_texts, maxlen=max_sequence_length, padding='post', truncating='post')
    features = feature_extractor_model.predict(padded_texts)
    return features

In [ ]:
from tensorflow.keras.models import Model

# Remove the second-to-last layer (the dropout layer)
feature_extractor_model2 = Model(inputs=model2.input, outputs=model2.layers[-3].output)

In [ ]:
from tensorflow.keras.models import Model

# Remove the second-to-last layer (the dropout layer)
feature_extractor_model3 = Model(inputs=model3.input, outputs=model3.layers[-3].output)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)  # Change the number of labels accordingly

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert_model.to(device)

checkpoint = torch.load("/content/drive/MyDrive/bert_hs.pth", map_location=device)
bert_model.load_state_dict(checkpoint)

bert_model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device("cpu")
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)

def extract_featuresb(text):

    text=preprocess_text(text)

    tokenized_text = bert_tokenizer.encode(text, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

    input_ids = torch.tensor([tokenized_text]).to(device)

    with torch.no_grad():
        bert_model.eval()
        outputs = bert_model(input_ids)
        encoder_outputs = outputs.last_hidden_state  # Accessing the last layer hidden states

    return encoder_outputs


In [ ]:
features1 = extract_features(df, feature_extractor_model, tokenizer)

print("Shape of features:", features1.shape)

16/16 [==============================] - 16s 705ms/step
Shape of features: (499, 800)


In [ ]:
features2 = extract_features(df, feature_extractor_model2, tokenizer)

print("Shape of features:", features2.shape)

16/16 [==============================] - 26s 233ms/step
Shape of features: (499, 800)


In [ ]:
features3 = extract_features(df, feature_extractor_model3, tokenizer)

print("Shape of features:", features3.shape)

16/16 [==============================] - 11s 496ms/step
Shape of features: (499, 800)


In [ ]:
texts = df['img_text']

# Extract features for each text and store them in a list
features_list = []
for text in texts:
    features = extract_featuresb(text)
    features_list.append(features)

print(features_list)
print(len(features_list))

[tensor([[[-0.4479,  0.5481,  0.5388,  ..., -0.5200,  0.5433, -0.5479],
         [ 0.5337, -0.0044,  0.6750,  ...,  0.0449,  0.5676, -0.8298],
         [ 0.9896, -0.1216,  0.2392,  ...,  0.0833,  0.1528, -0.8463],
         ...,
         [ 0.1040, -0.0641,  0.9205,  ..., -0.6478,  0.1281, -0.8146],
         [ 0.0409,  0.0898,  0.8336,  ..., -0.5941,  0.0071, -0.8278],
         [ 0.0332,  0.1812,  0.8333,  ..., -0.5616, -0.0396, -0.9490]]]), tensor([[[-4.6263e-01,  3.2242e-01,  4.5534e-01,  ..., -3.8851e-01,
           4.8456e-01, -7.5958e-01],
         [-1.7329e+00,  1.1787e-01,  6.6265e-01,  ..., -4.4282e-01,
           3.4582e-01, -1.4736e-01],
         [-1.3186e+00,  1.3232e-01,  1.4423e+00,  ..., -2.2194e-01,
           4.1114e-01, -7.2031e-01],
         ...,
         [ 1.4589e-01, -2.7737e-01,  8.8855e-01,  ..., -6.5015e-01,
           8.5441e-02, -8.7363e-01],
         [ 8.8807e-02, -1.0276e-01,  8.0468e-01,  ..., -5.8640e-01,
          -2.8040e-02, -8.9857e-01],
         [ 9.6909

In [ ]:
import numpy as np

# Assuming features_list contains the list of tensors

# Concatenate the tensors along a new dimension
concatenated_features = torch.cat(features_list, dim=0)

# Convert the concatenated tensor to a NumPy array
features_array = concatenated_features.cpu().numpy()

print(features_array)

[[[-4.47862476e-01  5.48126519e-01  5.38802147e-01 ... -5.20044506e-01
    5.43300688e-01 -5.47888875e-01]
  [ 5.33657193e-01 -4.39520134e-03  6.75010383e-01 ...  4.49351259e-02
    5.67649901e-01 -8.29771042e-01]
  [ 9.89592552e-01 -1.21568501e-01  2.39229783e-01 ...  8.33413005e-02
    1.52760163e-01 -8.46318901e-01]
  ...
  [ 1.04012959e-01 -6.40554652e-02  9.20517623e-01 ... -6.47760987e-01
    1.28070459e-01 -8.14610302e-01]
  [ 4.09327522e-02  8.98117498e-02  8.33573461e-01 ... -5.94052911e-01
    7.07163522e-03 -8.27826679e-01]
  [ 3.32348458e-02  1.81187749e-01  8.33270907e-01 ... -5.61630011e-01
   -3.96042019e-02 -9.49016333e-01]]

 [[-4.62631047e-01  3.22422087e-01  4.55336928e-01 ... -3.88505161e-01
    4.84558523e-01 -7.59575784e-01]
  [-1.73286998e+00  1.17866732e-01  6.62652016e-01 ... -4.42824155e-01
    3.45822603e-01 -1.47361502e-01]
  [-1.31863582e+00  1.32318482e-01  1.44233263e+00 ... -2.21944824e-01
    4.11141902e-01 -7.20309973e-01]
  ...
  [ 1.45893499e-01 -2.7

In [ ]:
# Compute the mean of the feature vectors for each text
mean_features_list = np.mean(features_array, axis=1)

print(mean_features_list)
print(len(mean_features_list))  # Ensure that the list contains 1000 entries
print(len(mean_features_list[0]))

[[ 0.06113915  0.162495    0.8825746  ... -0.6225503   0.05013294
  -1.0412523 ]
 [ 0.09888814 -0.03118183  0.85562474 ... -0.56790274  0.04266533
  -1.1727036 ]
 [ 0.08074226  0.05368773  0.8284811  ... -0.6060718   0.0092888
  -1.0962828 ]
 ...
 [ 0.1126665  -0.12173712  0.7302576  ... -0.5760341   0.02426636
  -1.0432221 ]
 [ 0.05692666 -0.20663048  0.7479615  ... -0.57340604  0.08219828
  -0.9120694 ]
 [ 0.08744884  0.19557323  0.78499395 ... -0.62488174  0.06346291
  -1.0311385 ]]
499
768


In [ ]:
import numpy as np

# Assuming mean_features_list, feature1, feature2, feature3 are arrays of 1000 entries each with 800 elements

mean_features_list = np.resize(mean_features_list, (499, 800))

print(mean_features_list)

# Compute the weighted average
weighted_features = 0.7 * mean_features_list + 0.1 * features1 + 0.1 * features2 + 0.1 * features3

print(weighted_features)
print(weighted_features.shape)  # Ensure that the resulting array contains 1000 entries each of 800 elements


[[ 0.06113915  0.162495    0.8825746  ...  0.32095292 -0.01030857
   0.11897967]
 [ 0.5436551  -0.0570736  -0.06995057 ...  0.7771015  -0.80341864
   0.04645481]
 [-0.46989548  0.732324   -0.2848815  ...  0.36633977 -0.20190042
   0.2971469 ]
 ...
 [ 0.23478612 -0.4697814  -0.76370233 ... -0.07084224 -0.25468853
   0.18474184]
 [ 0.2467947   0.1110702   0.38347486 ... -0.15129924  0.17920814
   0.9063047 ]
 [ 0.6229358  -0.44429776 -0.18889296 ...  0.16111945  0.8646763
   0.03006297]]
[[ 0.05087737  0.15793096  0.7135805  ...  0.32472926 -0.00753787
   0.08344164]
 [ 0.38863853  0.00423293  0.0468129  ...  0.64403325 -0.5627149
   0.03267424]
 [-0.32084686  0.5568112  -0.10363875 ...  0.35650006 -0.14165215
   0.20815869]
 ...
 [ 0.17243025 -0.28466251 -0.43881333 ...  0.05047265 -0.17860383
   0.12947515]
 [ 0.18083626  0.12193359  0.3642107  ... -0.00584725  0.12512383
   0.63456917]
 [ 0.444135   -0.266824   -0.03644675 ...  0.21284583  0.60495156
   0.02119995]]
(499, 800)


In [ ]:
print(len(weighted_features[0]))

800


In [ ]:
import numpy as np

# Save the array to a CSV file
np.savetxt('part3_weighted_features.csv', weighted_features, delimiter=',')

In [ ]:
df

,id,img_text,img_path,label
0,0,charlie sheen after his tigers blood kicks in,/content/drive/My Drive/Hateful-memes/data/img...,0
1,1,when you've finished with your life of crime,/content/drive/My Drive/Hateful-memes/data/img...,0
2,2,who would win? ching-chongs one eggy boi,/content/drive/My Drive/Hateful-memes/data/img...,1
3,3,meanwhile at the isis strip club,/content/drive/My Drive/Hateful-memes/data/img...,0
4,4,"enough with the nazi jokes, they're making me ...",/content/drive/My Drive/Hateful-memes/data/img...,0
...,...,...,...,...
494,494,nobody wants to hang auschwitz me,/content/drive/My Drive/Hateful-memes/data/img...,1
495,495,when god grants you a child after 20 years of ...,/content/drive/My Drive/Hateful-memes/data/img...,1
496,496,gays on social media: equality! body positivit...,/content/drive/My Drive/Hateful-memes/data/img...,1
497,497,having a bad day? you could be a siamese twin ...,/content/drive/My Drive/Hateful-memes/data/img...,1


In [ ]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-9s2n_8cs
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-9s2n_8cs
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [ ]:
import clip

# Get the list of available models
available_models = clip.available_models()

# Print the available models
print(available_models)


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [ ]:
import clip
from PIL import Image
import numpy as np

# Load the CLIP model and processor
model, preprocess = clip.load("ViT-B/32")


ModuleNotFoundError: No module named 'clip'

In [ ]:
# Function to extract features from an image
def extract_featuresi(image_path):
    # Load image
    image = Image.open(image_path)

    # Preprocess image
    image_input = preprocess(image).unsqueeze(0)

    # Forward pass through the model
    with torch.no_grad():
        image_features = model.encode_image(image_input)

    # Convert tensor to numpy array
    image_features = image_features.cpu().numpy()

    return image_features

In [ ]:
# Extract features for each image in img_array
features_list = []
for image_path in df['img_path'][:4000]:  # Considering 4000 entries
    features = extract_featuresi(image_path)
    features_list.append(features)

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame without headers
w = pd.read_csv("part3_weighted_features.csv", header=None)

# Display the DataFrame
print(w)


          0         1         2         3         4         5         6    \
0    0.050877  0.157931  0.713580  0.010365  0.129109 -0.227744  0.444341   
1    0.388639  0.004233  0.046813 -0.027773 -0.112698  0.217730  0.232952   
2   -0.320847  0.556811 -0.103639 -0.107517 -0.205580 -0.869055  0.147271   
3   -0.001310  0.065126 -0.272293 -0.354890 -0.375333  0.746782  0.311982   
4   -0.135240  0.477218  0.075390  0.127668 -0.098711  0.150167 -0.401842   
..        ...       ...       ...       ...       ...       ...       ...   
494  0.149635  0.182981 -0.111240  0.034236  0.219128 -0.438370  0.363627   
495  0.186534  0.368822  0.147354  0.417798  0.257255  0.151404  0.365545   
496  0.172430 -0.284663 -0.438813  0.005433 -0.093107 -0.019834 -0.065302   
497  0.180836  0.121934  0.364211  0.435060 -0.321626 -0.104927  0.288741   
498  0.444135 -0.266824 -0.036447  0.130761  0.074152 -0.269357  0.472297   

          7         8         9    ...       790       791       792  \
0  

In [ ]:
import pandas as pd
import numpy as np

# Reshape the arrays in features_list
reshaped_features_list = [arr.reshape(-1) for arr in features_list]

# Create a DataFrame
features_df = pd.DataFrame(reshaped_features_list)

# Check the shape
print(features_df.shape)

(499, 512)


In [ ]:
import pandas as pd

# Assuming `w` and `features_df` are your DataFrames
combined_df = pd.concat([w, features_df], axis=1)

# Display the combined DataFrame
print(combined_df)


          0         1         2         3         4         5         6    \
0    0.050877  0.157931  0.713580  0.010365  0.129109 -0.227744  0.444341   
1    0.388639  0.004233  0.046813 -0.027773 -0.112698  0.217730  0.232952   
2   -0.320847  0.556811 -0.103639 -0.107517 -0.205580 -0.869055  0.147271   
3   -0.001310  0.065126 -0.272293 -0.354890 -0.375333  0.746782  0.311982   
4   -0.135240  0.477218  0.075390  0.127668 -0.098711  0.150167 -0.401842   
..        ...       ...       ...       ...       ...       ...       ...   
494  0.149635  0.182981 -0.111240  0.034236  0.219128 -0.438370  0.363627   
495  0.186534  0.368822  0.147354  0.417798  0.257255  0.151404  0.365545   
496  0.172430 -0.284663 -0.438813  0.005433 -0.093107 -0.019834 -0.065302   
497  0.180836  0.121934  0.364211  0.435060 -0.321626 -0.104927  0.288741   
498  0.444135 -0.266824 -0.036447  0.130761  0.074152 -0.269357  0.472297   

          7         8         9    ...       502       503       504  \
0  

In [ ]:
# Save the array to a CSV file
np.savetxt('part3_clip_combined_features.csv', combined_df, delimiter=',')